<a href="https://colab.research.google.com/github/Laurene-v/Poem_generation_through_phonetics/blob/main/GPT_2_rhyme_generation_phonetics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Train a GPT-2 Text-Generating Model w/ GPU 

Adapted from [Max Woolf](http://minimaxir.com)



In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
import os

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



## Downloading GPT-2 ("small" version)

Here, I am retraining the GPT-2 model on phonetic transcriptions.


In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 580Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 5.04Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 294Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:09, 54.3Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 237Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 6.60Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 5.22Mit/s]


In [ ]:
 gpt2.mount_gdrive()

Mounted at /content/drive


## Uploading a Text File to be Trained to Colaboratory


In [ ]:

wdir = "/content/drive/MyDrive/data"
os.chdir(wdir)


In [ ]:
# Main datasets used :

#file_name = "adele.txt"
#file_name = "adele_phonetics.txt"
#file_name = "merged_lyrics.txt"
#file_name = "merged_lyrics_phonetics.txt"
#file_name = "merged_quatrains_cleaned.txt"
file_name = "merged_quatrains_cleaned_phonetics.txt"
#file_name = "same_two_rhymes_cleaned_phonetics.txt"

gpt2.copy_file_from_gdrive(file_name)

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

(The training might time out after 4ish hours)

The runtime needs to be restarted every time this cell is ran.

Other optional-but-helpful parameters for `gpt2.finetune`:

*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run_quatrains',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


100%|██████████| 1/1 [00:00<00:00, 126.28it/s]

Loading dataset...
dataset has 17265 tokens
Training...


[10 | 29.20] loss=2.54 avg=2.54
[20 | 50.85] loss=2.19 avg=2.36
[30 | 72.76] loss=2.08 avg=2.27
[40 | 94.89] loss=1.91 avg=2.18
[50 | 117.22] loss=1.79 avg=2.10
[60 | 139.73] loss=1.64 avg=2.02
[70 | 162.34] loss=1.52 avg=1.95
[80 | 184.88] loss=1.37 avg=1.87
[90 | 207.42] loss=1.14 avg=1.79
[100 | 229.96] loss=0.87 avg=1.69
[110 | 252.53] loss=0.87 avg=1.61
[120 | 275.10] loss=0.65 avg=1.53
[130 | 297.66] loss=0.54 avg=1.45
[140 | 320.21] loss=0.26 avg=1.36
[150 | 342.76] loss=0.17 avg=1.27
[160 | 365.29] loss=0.09 avg=1.19
[170 | 387.83] loss=0.08 avg=1.12
[180 | 410.36] loss=0.06 avg=1.06
[190 | 432.90] loss=0.07 avg=1.00
[200 | 455.44] loss=0.06 avg=0.95
======== SAMPLE 1 ========
rniv, sɛˈlɔnd dɪd mɔrn
ˈprɑməs laɪf ɪz nɑt kɔz rɪˈmɛmbərd
hi ˈɔlˌweɪz wɪθ reɪn aɪ dɪd nɑt steɪpt aɪ du gɪrd

hi bɪˈkeɪm ju tɔrn

hi bɪˈkeɪm tɔrn

hi ˈsəmˌwən wɪl goʊ wɪθ maɪ ləv
bət ˈwɪntɪd tɪ ju əˈləv

hi pɪkt əˈpɪkt tɪ sɪˈdeɪ, ənd aɪ deɪ
ɪn 1886 - 1971 ðə ˈsɪti kaɪnd
ɪt sɛnd ɪn ðɪs ˈwərldli nɔr
əˈveɪm ð

Copying the checkpoint folder to Google drive :

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run_wikisent')

If starting from a saved checkpoint : (same problem: if these cells are ran more than once, the runtime needs to be restarted)

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

## Generate Text From The Trained Model


In [ ]:
gpt2.generate(sess, run_name='run_wikisent')

npmi]ˈsɛmbərˈtænˈɛʤənfərsɪˈzɛˈtɑn

npmi]ˈsɛmbərˈtænˈɛʤənfərsɪˈzɛˈtɑn
npmi]ˈsɛmbərˈtænˈɛʤənfərsɪˈzɛˈtɑn

npmi]ˈsɛmbərˈtænˈɛʤənfərsɪˈzɛˈtɑn
ˈæftərˈsɛmbərˈtænˈɛʤənfərsɪˈzɛˈtɑn
ˈæftərˈtænˈɛʤənfərsɪˈzɛˈtɑn

ˈæftərˈtænˈɛʤənfərsɪˈzɛˈtɑn
ˈæftərˈtænˈɛʤənfərsɪˈzɛˈtɑn
ˈæftərˈtænˈɛʤənfərsɪˈzɛˈtɑn
ˈæftərˈtænˈɛʤənfərsɪˈzɛˈtɑn
ˈæftərˈtænˈɛʤənfərsɪˈzɛˈtɑn

ˈæftərˈtænˈɛʤənfərsɪˈzɛˈtɑn
ˈæftərˈtænˈɛʤənfərsɪˈzɛˈtɑn
ˈæftərˈtænˈɛʤənfərsɪˈzəˈmɑn
ˈæftərˈtænˈɛʤənfərsɪˈzɪrˈvɛktərˈɑn

ˈæftərˈtænˈɛʤənfərsɪˈzɑn
ˈæftərˈtænˈɛʤənfərsɪˈzəˈmɛktərˈɑn

ˈæftərˈtænˈɛʤənfərsɪˈzɑn
ˈæftərˈtænˈɛʤənfərsɪˈzɑn
ˈæftərˈtænˈɛʤənfərsɪˈzɛˈtɑn

ˈæftərˈtænˈɛʤənfərsɪˈzɪrˈvɛktərˈɑn
ˈæftərˈtænˈɛʤənfərsɪˈzɪrˈvɛktərˈɑn
ˈæftərˈtænˈɛʤənfərsɪˈzɪrˈvɛktərˈɑn

ˈæftərˈtænˈɛʤənfərsɪˈzɪrˈvɛktərˈɑn
ˈæftərˈtænˈɛʤənfərsɪˈzɪrˈvɛktərˈɑn
ˈæftərˈtænˈɛʤənfərsɪˈzɪrˈvɛktərˈɑn

ˈæftərˈtænˈɛʤənfərsɪˈzɪrˈvɛktərˈɑn
ˈæftə



Parameters for `gpt2.generate`:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
gpt2.generate(sess,
              length=1000,
              temperature=0.9,
              prefix="aɪ",
              nsamples=5,
              batch_size=5
              )

aɪt ˈpɑndər
wi kən du wɪˈθaʊt moʊst θɪŋz ɪn laɪf
bət ɪt wʊd ˌɪnˈdid bi fʊl əv straɪf
ɛz wi can’t* du wɪˈθaʊt ə ˈfɑrmər

ˈhɪstəri əv ðə meɪ deɪ
fər ðə ˈfæktəri ˈwərkər
frɛʃ bləd hu dɪd peɪ
ɪn 1886 ðət ˈvɛri jɪr

fər wɛl ˈwərkɪŋ pleɪs
eɪt aʊərz pər deɪ ˈduti
wɛl ənd ˈdɪgnəˌfaɪd weɪʤ
ənd fər ˈsərvɪs sɪˈkjʊrəti

ju ər ˈmɛni
hum wɪl aɪ kɔl
baɪ wən kɔl kəmz hi
ənd baɪ əˈnəðər ʃi

ðɛr ɪz ˈsəmθɪŋ greɪt tɪ ˈgɑrdənɪŋ
ənd ɪn ðiz ˈərli deɪz əv spərɪŋ
səm bərdz du ˈfrɑlɪk ənd ˈflətər
waɪl ˈəðərz du sɪŋ

ðə skaɪ ɪz ˈskætərd ənd ˈfɑlən daʊn
ðə ərθ ɪz noʊ mɔr braʊn
ˈwɪntər həz ˈbɪdɪn gʊd baɪ tɪ ðə fɔl
snoʊ fleɪks hæv ˈwoʊvən ðə waɪt gaʊn

ˈvɛri sun ðeɪ hæv ˈstɑrtɪd tɪ krɔl
ɔn ðə bɛr triz ˈstændɪŋ tɔl
wɪθ ˈkrɪsməs triz, wɪn ðeɪ əˈbaɪd
iʧ lʊks laɪk ə ˈkɔtən ˈkændi bɔl

wɪθ stɔrd fud, ˈhævɪŋ ˈfistɪd ˌɪnˈsaɪd
ɪn ðɛr nɛsts, ðə skwərəlz laɪk tɪ haɪd
sən ɪz tu maɪld tɪ dɪˈzɑlv ðə wɔrmθ
ərθ ɪz drɛst laɪk ə ˈwɛdɪŋ braɪd

trit ðə truθ ɛz truθ
ənd ˈfælsˌhʊd ɛz ˈfælsˌhʊd
du nɑt mɪks wən wɪθ ˈəðər
du nɑt bi ˈ
aɪn
